# WSI - ćwiczenie 1.
### Zagadnienie przeszukiwania i podstawowe podejścia do niego


In [1]:
import numpy as np
import plotly.graph_objects as go

1. Narysować funkcje f(x) i g(x).
2. Zaimplementować algorytm najszybszego spadku oraz zastosować go do znalezienia minimum
funkcji f i g.
3. Zbadać wpływ rozmiaru kroku dla różnych (losowych) punktów początkowych.

In [18]:
def f(x):
    return 10*(x**4) + 3*(x**3) - 30*(x**2) + 10*x 
    
def g(vect):
    assert len(vect) == 2
    return 10*(vect[1]**4) + 10*(vect[0]**4) + 3*(vect[0]**3) - 30*(vect[0]**2) + 10*vect[0]

In [36]:
# create 3D plot
fig = go.Figure()
# add trace for each scale step
for step in np.arange(1, 10, 1):
    X, Y = 2*[np.linspace(-step, step, 100)]
    fig.add_trace(go.Surface(z=[[g([x, y]) for y in Y] for x in X], 
                             x=X, 
                             y=Y, 
                             visible=False))
# make first trace visible
fig.data[0].visible = True

# create and add slider
steps = []
for i in range(len(fig.data)):
    step = dict(
        method="update",
        args=[{"visible": [False] * len(fig.data)}])
    step["args"][0]["visible"][i] = True
    steps.append(step)

sliders = [dict(
    active=10,
    steps=steps
)]

fig.update_layout(sliders=sliders, title='Plot of g(x, y)', autosize=False)
fig.show()
#plotly do wizualizacji kroków